In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import numpy as np
import pandas as pd
import matplotlib

In [2]:
df = pd.read_csv("data/contamination_fracking.csv")
df.dropna(inplace=True)

In [42]:
df.head()

,Unnamed: 0,level_0,CollectionDate,FacilityID,State,ContaminantCount,index,FacilityWaterType,Zip,LatitudePWS,...,PercentHighAdditive,PercentHFJob,IngredientName,Purpose,Supplier,JobDuration,TotalBaseWaterVolume,TotalBaseNonWaterVolume,distance_nearest,distance_20_nearest
1,1,13420,2014-02-06,1,CA,28,67540,SW,91802,33.786594,...,30.000000,1.729810,"('Ethoxylated Alcohol', 'Sodium Aryl Sulfonate...","('Surfactant', 'Non-emulsifier', 'Scale Inhibi...","('Baker Hughes', 'Other Chemicals', 'Operator')",1.0,287133.0,0.0,11.066041,11.722878
4,4,13423,2014-02-06,1,FL,28,89596,GW,32353,30.549676,...,36.573585,1.897712,"('Aluminum Oxide', 'Ethylene Glycol', 'Ethoxyl...","('Cross-linkers', 'Bulk Acid', 'Acid Corrosion...","('SAGA PETROLEUM', 'Nabors Completion and Prod...",0.0,53000.0,0.0,4.053320,7.282085
5,5,13424,2014-02-06,1,MA,21,55442,GW,1720,42.477142,...,27.013514,2.688290,"('Hydrochloric Acid', 'Alcohol Ethoxylate', 'C...","('Friction Reducer', 'Non-Emulsifier', 'Iron C...","('Chemplex', 'Reagent', 'TSS', 'Rain for Rent')",15.0,12878460.0,0.0,4.174925,4.437170
6,6,13425,2014-02-06,1,MD,26,50446,SW,21078,39.554644,...,32.940000,2.857143,('Potassium salt of diethylene triamine penta ...,"('Bulk Acid', 'Friction Reducer', 'Proppant', ...","('U.S. Well Services, LLC', 'Inflection Energy...",5.0,6132042.0,0.0,1.922339,2.061130
7,7,13426,2014-02-06,1,MN,21,39511,SW,55427,45.006764,...,23.654167,4.147837,"('Alkyl amine salts', '2,2-dibromo-3-nitrilopr...","('Scale Inhibitor', 'Flowback Enhancer', 'Othe...","(nan, 'Trican', 'Customer')",1.0,90509.0,0.0,8.230094,8.549826


In [10]:
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(df,df.ContaminantCount,test_size=0.3)

In [15]:
from sklearn.metrics import mean_squared_error

In [41]:
f = ['JobDuration','PercentHFJob','PercentHighAdditive','TotalBaseWaterVolume','distance_nearest','TotalBaseNonWaterVolume','SizeBool']
for x in f:
    reg = LinearRegression()
    reg.fit(np.array(X_train[x]).reshape(-1,1),Y_train)
    pred = reg.predict(np.array(X_test[x]).reshape(-1,1))
    print(x)
    print("MSE : ",mean_squared_error(pred,Y_test))
    print("R Score", reg.score(np.array(X_test[x]).reshape(-1,1),Y_test))
    print("Coeff:", reg.coef_)

reg = LinearRegression()
reg.fit(np.array(X_train[f]),Y_train)
pred = reg.predict(np.array(X_test[f]))
print("Combined")
print("MSE : ",mean_squared_error(pred,Y_test))
print("R Score", reg.score(np.array(X_test[f]),Y_test))
print("Coeff:", reg.coef_)

JobDuration
MSE :  78.92412855735515
R Score -3.57020340038261e-05
Coeff: [0.00192056]
PercentHFJob
MSE :  78.74275815311071
R Score 0.002262414940515045
Coeff: [-0.15116265]
PercentHighAdditive
MSE :  78.05545954113697
R Score 0.010971071754288686
Coeff: [-0.09589621]
TotalBaseWaterVolume
MSE :  78.88357369823817
R Score 0.00047816245521214995
Coeff: [-3.40238067e-08]
distance_nearest
MSE :  77.35304417948007
R Score 0.01987127061810723
Coeff: [0.14426051]
TotalBaseNonWaterVolume
MSE :  78.76590556812756
R Score 0.001969117544018295
Coeff: [-1.77087384e-07]
SizeBool
MSE :  78.74621555678189
R Score 0.002218606701251402
Coeff: [1.24949175]
Combined
MSE :  76.45073713747979
R Score 0.03130426674604325
Coeff: [ 4.88349289e-03  9.01647706e-02 -9.91864264e-02 -3.54816615e-08
  1.30783066e-01 -5.48004344e-08  1.31312071e+00]


In [44]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

for x in ['FacilityWaterType','Zip']:
    enc.fit(np.array(df[x]).reshape(-1,1))
    reg.fit(enc.transform(np.array(X_train[x]).reshape(-1,1)).toarray(), Y_train)
    en = enc.transform(np.array(X_test[x]).reshape(-1,1)).toarray()
    pred = reg.predict(en)
    print(x)
    print("MSE : ",mean_squared_error(pred,Y_test))
    print("R Score", reg.score(en,Y_test))
    print("Coeff:", reg.coef_)
    print("Max Coeff:",max(reg.coef_))

FacilityWaterType
MSE :  74.85630623337913
R Score 0.05150705970267733
Coeff: [-4.12155739e+13 -4.12155739e+13 -4.12155739e+13 -4.12155739e+13]
Max Coeff: -41215573899553.32
Zip
MSE :  5.569151958573542e+27
R Score -7.056588258150767e+25
Coeff: [9.66596017e+08 9.66596017e+08 9.66596017e+08 ... 9.66596009e+08
 9.66596003e+08 9.66596016e+08]
Max Coeff: 1426903606502695.8
